In [ ]:
import random
import numpy as np

from RobotEnviroment.init import *
from RobotEnviroment.arenas import RectangularArena
from PointClouds.PushFinder import *
from PointClouds.Registration import *
from PathPlanning.scanning import *
from PathPlanning.motion import *
from utils.debug import *

### Initialise Robot and Configuration

In [ ]:
ON_REAL = False
ROBOT_SPEED = .2 # m/s
INITIAL_OBJ_POS = np.array([-.50, -.1, .69])

allowedViewAngleSegments = [[0., np.pi*.25], [.75*np.pi, 1.25*np.pi], [1.75*np.pi, np.pi*2]]

C = setup_config(INITIAL_OBJ_POS, ON_REAL)
bot = startup_robot(C, ON_REAL)
visualizeViewAreas(C, INITIAL_OBJ_POS, allowedViewAngleSegments)

Define an arena to stop the robot from moving the object too far

In [ ]:
TABLE_CENTER = np.array([-.23, -.16, .651])
TABLE_DIMS = np.array([.89, .55])
ROBOT_POS = np.array([-.03, -.24, .651])
ROBOT_RING = .29

arena = RectangularArena(middleP=TABLE_CENTER, width=TABLE_DIMS[0], height=TABLE_DIMS[1], middlePCirc=ROBOT_POS, innerR=ROBOT_RING)
arena.plotArena(C)

### Main pushing loop
- Look towards the predicted object position
- Get the object's point cloud and calculate its normals
- Choose a random pushing point in the point cloud
- Calculate the pushing motion
- Push if the motion is feasible

In [ ]:
NUMBER_OF_PUSH_TRIALS = 10
predObjPos = INITIAL_OBJ_POS

pointClouds = [] # Stores the world position of the point cloud and the point cloud {"world_position": [x, y, z], "pc": np.array([])}
minNumScans = 2 # Minimum nuber of point cloud scans until we start merging them
fullPC = np.array([])

for i in range(NUMBER_OF_PUSH_TRIALS):

    print("Starting Trial Number ", i+1)

    # Scan object at it's predicted position and get it's possible push points
    lookAngle = giveRandomAllowedAngle(allowedViewAngleSegments)
    lookAtObjFromAngle(predObjPos, bot, C, 0., speed=ROBOT_SPEED) # Alternative: lookAtObj(predObjPos, bot, C)
    predObjPos, pointCloud = getScannedObject(bot, C, arena)
    if not len(predObjPos):
        print ("Lost the Object!")
        break
    push_points  = getPushPoints(pointCloud, verbose=0)

    # Store the object's point cloud
    for j, _ in enumerate(pointCloud):
        pointCloud[j] -= predObjPos
    pointClouds.append(pointCloud)

    if len(pointClouds) >= minNumScans:
        fullPC = joinOffsetPCS(pointClouds.copy(), verbose=0)

    # Try to push object
    for _ in range(5): # This counts the number of attempts to push an object from the current view angle. If it reaches 5 we try a new angle
        if not len(push_points):
            print("No viable push points found!")
            break

        pushP = random.choice(push_points)
        # push_points.remove(pushP)

        waypoints = pushMotionWaypoints(pushP[0], pushP[1], predObjPos, config=C)

        success = doPushThroughWaypoints(C, bot, waypoints, verbose=2, speed=ROBOT_SPEED)
        if success:
            break

In [ ]:
bot.home(C)

In [ ]:
del bot
del C